In [2]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder #, SimpleImputer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')
     

# Carregando o dataset
# URL do conjunto de dados Titanic
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

# Importando o conjunto de dados
df = pd.read_csv(url)
df = df.rename(columns={'Survived': 'Target'})
df = df.drop(columns=['PassengerId', 'Name', 'Ticket'])

# Selecionando as variáveis
features = [x for x in list(df.columns) if x not in ['Target']] # Exclui a coluna target
target = ['Target']

# Pré-processamento
# Imputação de valores missing
colunas_numericas = list(df.select_dtypes(include=['number']).columns)
colunas_numericas.remove('Target')
imputer = SimpleImputer(strategy='median')

# imputer.fit(df[features])

# Transformação de variáveis categóricas
categorical_features = list(df.select_dtypes(include=['category', 'object']).columns)
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# onehot_encoder.fit(df[categorical_features])

# Pré-processamento em pipeline
preprocessor = ColumnTransformer([
    ('imputer', imputer, colunas_numericas),
    ('onehot', onehot_encoder, categorical_features)
])
df_processado = pd.DataFrame(preprocessor.fit_transform(df[features]))


In [ ]:
!pip install boruta
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
###initialize Boruta
 
def boruta(X,y):
  forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5)
  boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
                  )
### fit Boruta (it accepts np.array, not pd.DataFrame)
  boruta.fit(np.array(X), np.array(y))
### print results
  green_area = X.columns[boruta.support_].to_list()
  blue_area = X.columns[boruta.support_weak_].to_list()
  print('features in the green area:', green_area)
  print('='*80)
  print('features in the blue area:', blue_area)
  return green_area
 
 
 
 
boruta_feauter = boruta(X = X, y = y)

In [3]:
# Seleção de variaveis utilizando o feature importance 
# feature_importances_ é quanato a variavel foi utilizada
# 
from sklearn.ensemble import RandomForestClassifier
X = df_processado
y = df['Target']

rfc = RandomForestClassifier(random_state=42).fit(X, y)

df_imp = \
(pd.DataFrame(list(zip(X.columns, rfc.feature_importances_)),
              columns=['feature_name', 'feature_importance'])
 .sort_values(by='feature_importance', ascending=False)
 .reset_index(drop=True)
)

df_imp

,feature_name,feature_importance
0,1,0.218110
1,4,0.206602
2,6,0.139361
3,5,0.114807
4,0,0.066755
...,...,...
154,72,0.000055
155,48,0.000031
156,111,0.000019
157,50,0.000016


In [12]:
from sklearn.inspection import permutation_importance

# Calcular importância de permutação
perm_importance = permutation_importance(rfc, X, y, n_repeats=10, random_state=42)

# Exibir importância de permutação das features
df_permut_imp = \
(pd.DataFrame(list(zip(X.columns, perm_importance.importances_mean)),
              columns=['feature_name', 'feature_permutation_importance'])
 .sort_values(by='feature_permutation_importance', ascending=False)
 .reset_index(drop=True)
)

df_permut_imp


,feature_name,feature_permutation_importance
0,1,0.125926
1,4,0.114703
2,6,0.106958
3,0,0.081818
4,2,0.046240
...,...,...
154,71,0.000000
155,72,0.000000
156,61,0.000000
157,103,-0.000112


In [70]:
# !pip install shap
import shap
explainer = shap.TreeExplainer(rfc)
shap_vals = explainer.shap_values(X)
# revisar  se é a media de tudo ou a media do primeiro resultado 
df_imp_shap = \
(pd.DataFrame(list(zip(X.columns, np.abs(shap_vals[0]).mean(axis=1))),  # np.abs(shap_vals).mean(axis=0).mean(axis=1))
              columns=['feature_name', 'shap_importance'])   
 .sort_values(by='shap_importance', ascending=False)
  .reset_index(drop=True)
)

df_imp_shap


,feature_name,shap_importance
0,6,0.085388
1,5,0.067428
2,4,0.061784
3,2,0.054476
4,0,0.032263
...,...,...
154,46,0.000016
155,72,0.000014
156,50,0.000013
157,41,0.000012


In [96]:

from boruta import BorutaPy
X = df_processado
y = df['Target']

rfc = RandomForestClassifier(
   n_jobs = -1, 
   max_depth = 5)

boruta = \
(BorutaPy(
    estimator=rfc,
    n_estimators=50,
    max_iter=100, # number of trials to perform
    random_state=42)
 ) # fit accepts np.array, not pd.DataFrame

boruta.fit(np.array(X), np.array(y))



AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [99]:
np.array(y)

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [97]:
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
import numpy as np
###initialize Boruta
 
def boruta(X,y):
  forest = RandomForestRegressor(
   n_jobs = -1, 
   max_depth = 5)
  boruta = BorutaPy(
   estimator = forest, 
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
                  )
### fit Boruta (it accepts np.array, not pd.DataFrame)
  boruta.fit(np.array(X), np.array(y))
### print results
  green_area = X.columns[boruta.support_].to_list()
  blue_area = X.columns[boruta.support_weak_].to_list()
  print('features in the green area:', green_area)
  print('='*80)
  print('features in the blue area:', blue_area)
  return green_area
 
 
 
 
boruta_feauter = boruta(X = X, y = y)

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [73]:
# Adaptar o atributo .feature_importances_ para o SHAP
class SHAPImportanceRandomForestClassifier(RandomForestClassifier):
    def fit(self, X, y, sample_weight=None):
        self.X_ = X
        super().fit(X, y, sample_weight=sample_weight)
        return self
    @property
    def feature_importances_(self):
        check_is_fitted(self)
        explainer = shap.TreeExplainer(self)
        shap_vals = explainer.shap_values(self.X_)
        return np.abs(shap_vals[0]).mean(axis=0)
    
# from shap_feature_importances_ import SHAPImportanceRandomForestClassifier

rfc_shap = SHAPImportanceRandomForestClassifier(random_state=42).fit(X, y)
rfc_shap.feature_importances_

NameError: name 'check_is_fitted' is not defined

In [84]:
from sklearn.base import BaseEstimator
from sklearn.feature_selection import SelectorMixin

class SelectKTop(SelectorMixin, BaseEstimator):

    def __init__(self, K=5, base_estimator=None, random_state=None):
        self.K = K
        self.base_estimator = base_estimator
        self.random_state = random_state

    def _validate_estimator(self, default):
        if self.base_estimator is not None:
            self.base_estimator_ = self.base_estimator
        else:
            self.base_estimator_ = default

    def fit(self, X, y):

        X = self._validate_data(
            X,
            dtype=np.float64,
        )

        self._validate_estimator(RandomForestClassifier(random_state=self.random_state))

        self.base_estimator_.fit(X, y)

        self._selected_index_list_ = \
        (pd.DataFrame(list(zip(range(self.n_features_in_), self.base_estimator_.feature_importances_)),
                      columns=['feature_name', 'feature_importance'])
         .sort_values(by='feature_importance', ascending=False)
         .head(self.K)
         .index
         .to_list()
        )

        return self

    def _get_support_mask(self):
        check_is_fitted(self)
        return np.array([feat in self._selected_index_list_ for feat in range(self.n_features_in_)])

In [85]:
import SelectKTop

from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold
from sklearn.pipeline import make_pipeline

grid = (
    GridSearchCV(
        make_pipeline(SelectKTop(random_state=42),
                      RandomForestClassifier(random_state=42)),
        param_grid={'selectktop__K': np.arange(1,N_FEATURES+1)},
        cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42),
        scoring='roc_auc')
    .fit(X, y))

df_cv = (
    pd.DataFrame(grid.cv_results_)[[
        'param_selectktop__K',
        'mean_test_score',
        'std_test_score'
    ]])

cv_best = (
    df_cv
    .sort_values(by='mean_test_score', ascending=False)
    .reset_index(drop=True)
    .loc[0])
plt.errorbar(df_cv.param_selectktop__K, df_cv.mean_test_score, 1.96*df_cv.std_test_score)
plt.scatter(cv_best.param_selectktop__K, cv_best.mean_test_score, s=100)
plt.ylim(0.75, 1)
plt.xlabel('K of SelectKTop')
plt.xticks(df_cv.param_selectktop__K.astype(int))
plt.ylabel('Performance (ROCAUC)')
plt.show()

ModuleNotFoundError: No module named 'SelectKTop'